In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
# all targets are categorical
def target_columns(version):
    return [
          f'cve.metrics.cvssMetricV{version}.cvssData.attackVector',
          f'cve.metrics.cvssMetricV{version}.cvssData.attackComplexity',
          f'cve.metrics.cvssMetricV{version}.cvssData.privilegesRequired',
          f'cve.metrics.cvssMetricV{version}.cvssData.userInteraction',
          f'cve.metrics.cvssMetricV{version}.cvssData.scope',
          f'cve.metrics.cvssMetricV{version}.cvssData.confidentialityImpact',
          f'cve.metrics.cvssMetricV{version}.cvssData.integrityImpact',
          f'cve.metrics.cvssMetricV{version}.cvssData.availabilityImpact'
      ]

In [4]:
cves = pd.read_csv('data/raw_cves.csv')
cves.shape

C:\Users\elyes\AppData\Local\Temp\ipykernel_11380\2092503873.py:1: DtypeWarning: Columns (13,19,20,21,22,23,40,41,44,45,46,47,48,52,53,54,55,56,57,58,59,60,62,63,84) have mixed types. Specify dtype option on import or set low_memory=False.
  cves = pd.read_csv('data/raw_cves.csv')


(197755, 87)

In [5]:
feature = ['cve.descriptions.lang.en']
target_31 = target_columns('31')

v31 = cves[cves['cve.metrics.cvssMetricV31']==True][target_31 + feature]

print( v31.shape)

(81889, 9)


In [24]:
for target in target_31:
    label_name = target[35:]
    
    df = v31[["cve.descriptions.lang.en",target]]
    
    df.rename(columns={"cve.descriptions.lang.en": "text", target: "labels"}, inplace=True)

    df['text'] = df['text'].astype(str)
    
    le = LabelEncoder()
    df['labels'] = le.fit_transform(df['labels'])

    X_train, X_test, y_train, y_test = train_test_split(df['text'], df['labels'], test_size=0.2, random_state=10)
    train_df = pd.concat([X_train, y_train], axis=1)
    test_df = pd.concat([X_test, y_test], axis=1)
    
    FolderExists = os.path.exists(f"data/{label_name}")
    if not FolderExists:
        os.makedirs(f"data/{label_name}")
    
    train_df.to_csv(f'data/{label_name}/train.csv',index=False)
    test_df.to_csv(f'data/{label_name}/test.csv',index=False)
    
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

    with open(f'data/{label_name}/label_mapping.txt', "w") as fp:
        fp.write(str(le_name_mapping))

C:\Users\elyes\AppData\Local\Temp\ipykernel_11380\1755501500.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"cve.descriptions.lang.en": "text", target: "labels"}, inplace=True)
C:\Users\elyes\AppData\Local\Temp\ipykernel_11380\1755501500.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].astype(str)
C:\Users\elyes\AppData\Local\Temp\ipykernel_11380\1755501500.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se